In [1]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.0 MB/s eta 0:00:00


In [2]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [4]:
from transformers import AutoTokenizer, DistilBertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [5]:
import json
with open('coco-classes.json') as f:
  coco_classes = json.load(f)

In [6]:
coco_classes

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

**hypothesis**: the context that BERT accepts along with the word may be helpful in generating higher quality embedding for this word.

**idea**: for each word in coco classes, use GPT4 to generate two sentences:
* Sentence A should use the word in a context that is very **specific and characteristic** of it, such as their behavior or traits. *(e.g., The **cat** purred contentedly as it curled up in the sunlight.)*
* Sentence B should also use the word, but in a more **general** or broader context where the presence or mention of it is not central to the sentence's meaning. *(e.g., There's a **cat** picture hanging in the hallway.)*


Compare the word embedding for the word without any context (using the word itself as the input) and with sentence A and setence B as the context to see whether these are stronger embeddings that help downstream tasks such as classifications and predictions.


In [49]:
from openai import OpenAI
client = OpenAI(
    api_key= "KEY",
)

In [8]:
def generateSentences(word):
  chat_completion = client.chat.completions.create(
      messages=[{"role": "user", "content": f"Please provide two sentences using the word '{word}'. Sentence A should use '{word}' in a context that is very specific and characteristic of {word}, reflecting behaviors or traits. Sentence B should also use '{word},' but in a more general or broader context where the presence or mention of a {word} is not central to the sentence's meaning. Make the sentence brief (10 words max). Format your response like this: A: '...' ; B: '...'."}],
      model="gpt-4",
  )
  sentences = chat_completion.choices[0].message
  return sentences

In [9]:
def getSentences(res):
  if ';' in res:
    parts = res.split(' ; ')
  else:
    parts = res.split('\n')
  sentence_a = parts[0].split("A: '")[1].rstrip("'")
  sentence_b = parts[1].split("B: '")[1].rstrip("'")
  return sentence_a, sentence_b

In [98]:
word = 'pineapple'
res = generateSentences(word).content

In [99]:
input_text_list = [word]
input_text_list.append(getSentences(res)[0])
input_text_list.append(getSentences(res)[1])

In [100]:
#input_text_full = []
input_text_full.append(input_text_list)

In [105]:
input_text_full

[['apple',
  'The apple has a juicy, crisp texture and sweet taste.',
  'He picked up an apple on his way to school.'],
 ['orange',
  'The ripe orange burst with juicy sweetness as she bit into it.',
  'She brings the orange home.'],
 ['banana',
  "The ripe banana was soft and sweet to taste.' ",
  'He brings a banana during our morning meeting.'],
 ['peach',
  'She bit into the ripe peach, tasting the sweet juices.',
  'He never liked how the peach smells.'],
 ['pineapple',
  "The pineapple's sweet tang perfectly offset the savory ham.",
  'On the table, amid the array of things, sat a pineapple.']]

In [52]:
# a function to get the embedding for a specific word
def getEmbedding(input, word):
    encoded_input = tokenizer(input, return_tensors = 'pt')
    with torch.no_grad():
      output = model(**encoded_input)
    token_embeddings = output.last_hidden_state
    token_ids_word = tokenizer.encode(word, add_special_tokens=False)
    word_token_positions = [i for i, token_id in enumerate(encoded_input['input_ids'][0]) if token_id in token_ids_word]
    word_embedding = token_embeddings[0, word_token_positions, :].mean(dim=0)
    # Retrieve the tokens for the identified positions and convert them back to text
    retrieved_tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0][word_token_positions])
    retrieved_word = tokenizer.convert_tokens_to_string(retrieved_tokens)

    # Print the original word, its tokens, and the retrieved word
    print(f"Original word: {word}")
    print(f"Tokens: {tokenizer.convert_ids_to_tokens(token_ids_word)}")
    return word_embedding

In [106]:
emb_list_full = []
for input_text_list in input_text_full:
  emb_list = []
  for input in input_text_list:
    emb = getEmbedding(input,input_text_list[0])
    emb_list.append(emb)
  emb_list_full.append(emb_list)


Original word: apple
Tokens: ['apple']
Original word: apple
Tokens: ['apple']
Original word: apple
Tokens: ['apple']
Original word: orange
Tokens: ['orange']
Original word: orange
Tokens: ['orange']
Original word: orange
Tokens: ['orange']
Original word: banana
Tokens: ['banana']
Original word: banana
Tokens: ['banana']
Original word: banana
Tokens: ['banana']
Original word: peach
Tokens: ['peach']
Original word: peach
Tokens: ['peach']
Original word: peach
Tokens: ['peach']
Original word: pineapple
Tokens: ['pine', '##apple']
Original word: pineapple
Tokens: ['pine', '##apple']
Original word: pineapple
Tokens: ['pine', '##apple']


In [107]:
with open('fruit_text_input.txt', 'w') as file:
    for item in input_text_full:
        file.write("%s\n" % item)

In [108]:
with open('fruit_text_emb.txt', 'w') as file:
    for item in emb_list_full:
        file.write("%s\n" % item)